In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv, dotenv_values 
import requests
import time
from bs4 import BeautifulSoup
load_dotenv()

True

In [2]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome()

driver.get("https://id.condenast.com/")
wait = WebDriverWait(driver, 15)

In [3]:
# The selector name may change when they update the website
vogue_login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#main-content > div > div.utility-card__grid.utility-card > div:nth-child(16) > a"))
)
vogue_login_button.click()

email = os.getenv("VOGUE_EMAIL")
password = os.getenv("VOGUE_PASSWORD")

email_input = wait.until(
        EC.presence_of_element_located((By.NAME, "email"))
)
email_input.send_keys(email)

email_continue_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#email-continue-button"))
)
# Need to use execute_script as it uses javascript, simulating a real user click 
driver.execute_script("arguments[0].click();", email_continue_button)

password_input = wait.until(
        EC.presence_of_element_located((By.NAME, "password"))
)
password_input.send_keys(password)

sign_in_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#log-in-sign-in-button-password"))
)
driver.execute_script("arguments[0].click();", sign_in_button)


no_passkey_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#do-not-setup-passkey-button"))
)
driver.execute_script("arguments[0].click();", no_passkey_button)

# It's probbaly to do with cookies as to why it's not working, need to save the cookies somewhere when I switch websites
# driver.execute_script("window.location.href='https://www.vogue.com/fashion-shows/spring-2026-ready-to-wear/christophe-lemaire/slideshow/collection#1'")
driver.get("https://www.vogue.com/fashion-shows/spring-2026-ready-to-wear/christophe-lemaire/slideshow/collection#1")



In [ ]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "img[data-src]"))
)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

✅ Saved lemaire_001.jpg


In [ ]:
def extract_image():
    # Extract all images with a data-src
    img_tag = soup.find_all("img", {"data-src": True})

    if img_tag:
        img_url = img_tag["data-src"]
        print("Image URL:", img_url)

        # Download the image
        headers = {"User-Agent": "Mozilla/5.0"}  # Pretend to be a browser
        img_data = requests.get(img_url, headers=headers).content

        filepath = os.path.join("images", "vogue_image.jpg")
        with open(filepath, "wb") as f:
            f.write(img_data)

        print("✅ Image saved as vogue_image.jpg")
    else:
        print("⚠️ No image found")